# Create Env
This script is heavily inspired by the [policy gradient implementation](https://github.com/simoninithomas/Deep_reinforcement_learning_Course/blob/master/Policy%20Gradients/Doom/Doom%20REINFORCE%20Monte%20Carlo%20Policy%20gradients.ipynb) in Thomas Simoninis RL-course 

In [ ]:
#!pip3 install tensorflow-gpu

In [1]:
import tensorflow as tf
import numpy as np
from vizdoom import *
import random
import time
from skimage import transform
from datetime import datetime

from collections import deque
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

In [2]:
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))
    
with tf.Session() as sess:
  devices = sess.list_devices()

print(devices)

[[22. 28.]
 [49. 64.]]
[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 12365308761443178060), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 5773923883061326042), _DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 9632874745224070731), _DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 5413076992, 8644213643834083865)]


In [3]:
def create_environment(visible=False):
    game = DoomGame()
    game.load_config("scenarios/deadly_corridor.cfg")
    game.set_window_visible(visible)
    game.init()
    possible_actions = np.identity(7, dtype=int).tolist()
    
    return game, possible_actions

In [4]:
game, possible_actions = create_environment(visible=False)

In [5]:
# INPUT SPECS
STACK_SIZE = 4
FRAME_SIZE = [84,84]

# ENV HYPERPARAMS
STATE_SIZE = FRAME_SIZE[:]; STATE_SIZE.append(STACK_SIZE)
ACTION_SIZE = game.get_available_buttons_size()
POSSIBLE_ACTIONS = np.identity(ACTION_SIZE, dtype=int).tolist()

# TRAINING HYPERPARAMS
LEARNING_RATE = 0.002
NUM_EPOCHS = 500000

BATCH_SIZE = 1000
GAMMA = 0.95 # Discounting rate

# Test env

In [6]:
def test_environment(episodes):
    game = create_environment(visible=True)
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice(POSSIBLE_ACTIONS)
            reward = game.make_action(action)
            time.sleep(0.007)
        print ("Result:", game.get_total_reward())
        time.sleep(1)
    game.close()

In [7]:
#test_environment(3)

# Define prepocessing functions

In [8]:
def preprocess_frame(frame):
    cropped_frame = frame[80:,:]
    normalized_frame = cropped_frame/255.0
    resized_frame = transform.resize(normalized_frame, FRAME_SIZE)
    return resized_frame

In [9]:
def init_deque():
    return deque([np.zeros(FRAME_SIZE, dtype=np.int) for i in range(STACK_SIZE)], maxlen=STACK_SIZE)

In [10]:
stacked_frames = init_deque()

In [11]:
def stack_frames(state, is_new_episode, stacked_frames = None):
    frame = preprocess_frame(state)
    
    if is_new_episode:
        stacked_frames = init_deque()
    else:
        stacked_frames.append(frame)
        
    stacked_state = np.stack(stacked_frames, axis=2)
    
    return stacked_state, stacked_frames

In [12]:
state = game.get_state().screen_buffer
state, stacked_frames = stack_frames(state, is_new_episode=True)

In [13]:
state.shape

(84, 84, 4)

# Discount_and_normalize_rewards
This function is important, because we are in a Monte Carlo situation. <br>

We need to **discount the rewards at the end of the episode**. This function takes, the reward discount it, and **then normalize them** (to avoid a big variability in rewards).

In [14]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_episode_rewards = np.zeros_like(episode_rewards)
    cumulative = 0.0
    for i in reversed(range(len(episode_rewards))):
        cumulative = cumulative * GAMMA + episode_rewards[i]
        discounted_episode_rewards[i] = cumulative
    
    mean = np.mean(discounted_episode_rewards)
    std = np.std(discounted_episode_rewards)
    discounted_episode_rewards = (discounted_episode_rewards - mean) / (std)

    return discounted_episode_rewards

# Create Policy Gradient Network model

In [15]:
# With a single method, should this even be a class?! Might have something
# to do with the way TensorFlow works? Or just simoninis noob-coding...
class PGNetwork:
    def __init__(self, name='PGNetwork'):
        
        with tf.variable_scope(name):
            with tf.name_scope("inputs"):
                tempArray = [None]
                for i in STATE_SIZE:
                    tempArray.append(i)
                
                self.inputs_= tf.placeholder(tf.float32, tempArray, name="inputs_")
                self.actions = tf.placeholder(tf.int32, [None, ACTION_SIZE], name="actions")
                self.discounted_episode_rewards_ = tf.placeholder(tf.float32, [None, ], name="discounted_episode_rewards_")
            
                
                # Variable for tensorboard
                self.mean_reward_ = tf.placeholder(tf.float32, name="mean_reward")
                
            with tf.name_scope("conv1"):
                """
                First convnet:
                CNN
                BatchNormalization
                ELU
                """
                # Input [84, 84, 4]
                self.conv1 = tf.layers.conv2d(inputs = self.inputs_,
                                             filters = 32,
                                             kernel_size = [8,8],
                                             strides = [4,4],
                                             padding = "VALID",
                                              kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                             name = "conv1")

                self.conv1_batchnorm = tf.layers.batch_normalization(self.conv1,
                                                       training = True,
                                                       epsilon = 1e-5,
                                                         name = 'batch_norm1')

                self.conv1_out = tf.nn.elu(self.conv1_batchnorm, name="conv1_out")
                # Output [20, 20, 32]
            
            with tf.name_scope("conv2"):
                """
                Second convnet:
                CNN
                BatchNormalization
                ELU
                """
                self.conv2 = tf.layers.conv2d(inputs = self.conv1_out,
                                     filters = 64,
                                     kernel_size = [4,4],
                                     strides = [2,2],
                                     padding = "VALID",
                                    kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                     name = "conv2")

                self.conv2_batchnorm = tf.layers.batch_normalization(self.conv2,
                                                       training = True,
                                                       epsilon = 1e-5,
                                                         name = 'batch_norm2')

                self.conv2_out = tf.nn.elu(self.conv2_batchnorm, name="conv2_out")
                # Output [9, 9, 64]
            
            with tf.name_scope("conv3"):
                """
                Third convnet:
                CNN
                BatchNormalization
                ELU
                """
                self.conv3 = tf.layers.conv2d(inputs = self.conv2_out,
                                     filters = 128,
                                     kernel_size = [4,4],
                                     strides = [2,2],
                                     padding = "VALID",
                                     kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                     name = "conv3")

                self.conv3_batchnorm = tf.layers.batch_normalization(self.conv3,
                                                       training = True,
                                                       epsilon = 1e-5,
                                                       name = 'batch_norm3')

                self.conv3_out = tf.nn.elu(self.conv3_batchnorm, name="conv3_out")
                # Output [3, 3, 128]
            
            with tf.name_scope("flatten"):
                self.flatten = tf.layers.flatten(self.conv3_out)
                # Output [1152]
            
            with tf.name_scope("fc1"):
                self.fc = tf.layers.dense(inputs = self.flatten,
                                          units = 512,
                                          activation = tf.nn.elu,
                                          kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                          name="fc1")
            
            with tf.name_scope("logits"):
                self.logits = tf.layers.dense(inputs = self.fc,
                                              units = ACTION_SIZE, 
                                              activation=None,
                                              kernel_initializer=tf.contrib.layers.xavier_initializer())
            
            with tf.name_scope("softmax"):
                self.action_distribution = tf.nn.softmax(self.logits)

            with tf.name_scope("loss"):
                self.neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.logits, labels = self.actions)
                self.loss = tf.reduce_mean(self.neg_log_prob * self.discounted_episode_rewards_) 
    
            with tf.name_scope("train"):
                self.train_opt = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(self.loss)

In [16]:
# Reset the graph
tf.reset_default_graph()

# Instantiate the PGNetwork
PGNetwork = PGNetwork()

# Initialize Session
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


# Setup Tensorboard

In [17]:
# Setup TensorBoard Writer
writer = tf.summary.FileWriter("/tensorboard/pg/test")

## Losses
tf.summary.scalar("Loss", PGNetwork.loss)

## Reward mean
tf.summary.scalar("Reward_mean", PGNetwork.mean_reward_ )

write_op = tf.summary.merge_all()

# Batch Function

Here we'll create batches.<br>
These batches contains episodes **(their number depends on how many rewards we collect**: for instance if we have episodes with only 10 rewards we can put BATCH_SIZE/10 episodes
<br>
* Make a batch
    * For each step:
        * Choose action a
        * Perform action a
        * Store s, a, r
        * **If** done:
            * Calculate sum reward
            * Calculate GAMMA Gt

In [18]:
def make_batch():
    states, actions, rewards_of_episode, rewards_of_batch, discounted_rewards = [], [], [], [], []

    episode_num = 1
    
    game.new_episode()
        
    state = game.get_state().screen_buffer
    state, stacked_frames = stack_frames(state, is_new_episode=True)

    while True:
        # Run state through policy and calculate action
        tempArray = [1]
        for i in STATE_SIZE:
            tempArray.append(i)
        #Det er i stedet for *STA stop! stop stop stop :) for i state_size?? virker det?
        action_probability_distribution = sess.run(PGNetwork.action_distribution, 
                                                   feed_dict={PGNetwork.inputs_: state.reshape(tempArray)})
        
        # Sample an action with respect to the action probabilities
        action = np.random.choice(range(action_probability_distribution.shape[1]), 
                                  p=action_probability_distribution.ravel())
        action = POSSIBLE_ACTIONS[action]

        # Perform sampled action
        reward = game.make_action(action)
        done = game.is_episode_finished()

        # Store results
        states.append(state)
        actions.append(action)
        rewards_of_episode.append(reward)
        
        if done:
            # The episode ends so no next state
            next_state = np.zeros(FRAME_SIZE, dtype=np.int)
            next_state, stacked_frames = stack_frames(next_state, is_new_episode=False, stacked_frames=stacked_frames)
            
            rewards_of_batch.append(rewards_of_episode)
            
            # Calculate gamma
            discounted_rewards.append(discount_and_normalize_rewards(rewards_of_episode))
           
            # If the number of rewards_of_batch > BATCH_SIZE stop the minibatch creation
            # (Because we have sufficient number of episode mb)
            # Remember that we put this condition here, because we want entire episode (Monte Carlo)
            # so we can't check that condition for each step but only if an episode is finished
            if len(np.concatenate(rewards_of_batch)) > BATCH_SIZE:
                break
                
            # Reset the transition stores
            rewards_of_episode = []
            
            episode_num += 1
            
            game.new_episode()

            state = game.get_state().screen_buffer
            state, stacked_frames = stack_frames(state, is_new_episode=True)
         
        else:
            # If not done, the next_state become the current state
            next_state = game.get_state().screen_buffer
            next_state, stacked_frames = stack_frames(next_state, is_new_episode=False, stacked_frames=stacked_frames)
            state = next_state
                         
    return np.stack(np.array(states)), np.stack(np.array(actions)), np.concatenate(rewards_of_batch), np.concatenate(discounted_rewards), episode_num

* Create the Neural Network
* Initialize the weights
* Init the environment
* all_rewards = 0 # Keep track of maximum reward
* **For** epochs in range(NUM_EPOCHS):
    * Get batches
    * Optimize


# Train agent

In [19]:
# Keep track of all rewards total for each batch
all_rewards = []



total_rewards = 0
maximum_reward_recorded = 0
mean_reward_total = [] # What is this?
epoch = 1
average_reward = []

saver = tf.train.Saver()

# Load the model
#saver.restore(sess, "./models/policy_model.ckpt")


#LOG
now = datetime.now()
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
with open("logs/runningTimePGN.txt", "a") as myfile:
    myfile.write("Starting:\n")
    myfile.write("date and time:" + date_time + "\n")



while epoch < NUM_EPOCHS + 1:
    # Gather training data
    states_mb, actions_mb, rewards_of_batch, discounted_rewards_mb, nb_episodes_mb = make_batch()

    ### These part is used for analytics
    # Calculate the total reward ot the batch
    total_reward_of_batch = np.sum(rewards_of_batch)
    all_rewards.append(total_reward_of_batch)

    # Calculate the mean reward of the batch
    # Total rewards of batch / nb episodes in that batch
    mean_reward_of_batch = np.divide(total_reward_of_batch, nb_episodes_mb)
    mean_reward_total.append(mean_reward_of_batch)

    # Calculate the average reward of all training
    # mean_reward_of_batch / epoch
    average_reward_of_all_training = np.divide(np.sum(mean_reward_total), epoch)

    # Calculate maximum reward recorded 
    maximum_reward_recorded = np.amax(all_rewards)

    print("==========================================")
    print("Epoch: ", epoch, "/", NUM_EPOCHS)
    print("-----------")
    #print("Number of training episodes: {}".format(nb_episodes_mb))
    #print("Total reward: {}".format(total_reward_of_batch, nb_episodes_mb))
    #print("Mean Reward of that batch {}".format(mean_reward_of_batch))
    #print("Average Reward of all training: {}".format(average_reward_of_all_training))
    #print("Max reward for a batch so far: {}".format(maximum_reward_recorded))

    
    # Feedforward, gradient and backpropagation
    loss_, _ = sess.run([PGNetwork.loss, PGNetwork.train_opt], feed_dict={PGNetwork.inputs_: states_mb.reshape((len(states_mb),84,84,4)),
                                                        PGNetwork.actions: actions_mb,
                                                                 PGNetwork.discounted_episode_rewards_: discounted_rewards_mb 
                                                                })
    
    
    #loss_, _ = sess.run([PGNetwork.loss, PGNetwork.train_opt], feed_dict={PGNetwork.inputs_: states_mb.reshape((len(states_mb), *STATE_SIZE)),
    #                                                    PGNetwork.actions: actions_mb,
    #                                                             PGNetwork.discounted_episode_rewards_: discounted_rewards_mb 
    #                                                            })

    print("Training Loss: {}".format(loss_))

    # Write TF Summaries
    summary = sess.run(write_op, feed_dict={PGNetwork.inputs_: states_mb.reshape((len(states_mb), 84,84,4)),
                                                        PGNetwork.actions: actions_mb,
                                                                 PGNetwork.discounted_episode_rewards_: discounted_rewards_mb,
                                                                PGNetwork.mean_reward_: mean_reward_of_batch
                                                                })

    #summary = sess.run(write_op, feed_dict={x: s_.reshape(len(s_),84,84,1), y:a_, d_r: d_r_, r: r_, n: n_})
    writer.add_summary(summary, epoch)
    writer.flush()

    # Save Model
    if epoch % 5 == 0:
        saver.save(sess, "./models/modelPGN.ckpt")
        print("Model saved")
        now = datetime.now()
        date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
        with open("logs/runningTimePGN.txt", "a") as myfile:
            myfile.write("date and time:" + date_time + "\n")
            myfile.write("Epoch: "+ str(epoch)+ "/"+ str(NUM_EPOCHS))
    epoch += 1

('Epoch: ', 1, '/', 500000)
-----------
Training Loss: 0.0548883974552
('Epoch: ', 2, '/', 500000)
-----------
Training Loss: 0.000150105421199
('Epoch: ', 3, '/', 500000)
-----------
Training Loss: 0.0104705868289
('Epoch: ', 4, '/', 500000)
-----------
Training Loss: 0.0529374480247
('Epoch: ', 5, '/', 500000)
-----------
Training Loss: -0.0223564039916
Model saved
('Epoch: ', 6, '/', 500000)
-----------
Training Loss: 0.00979884620756
('Epoch: ', 7, '/', 500000)
-----------
Training Loss: 0.00940455403179
('Epoch: ', 8, '/', 500000)
-----------
Training Loss: -0.00407387455925
('Epoch: ', 9, '/', 500000)
-----------
Training Loss: -0.00559436483309
('Epoch: ', 10, '/', 500000)
-----------
Training Loss: -0.022077281028
Model saved
('Epoch: ', 11, '/', 500000)
-----------
Training Loss: 0.0438382811844
('Epoch: ', 12, '/', 500000)
-----------
Training Loss: 0.0238537658006
('Epoch: ', 13, '/', 500000)
-----------
Training Loss: 0.00850224401802
('Epoch: ', 14, '/', 500000)
----------

Model saved
('Epoch: ', 71, '/', 500000)
-----------
Training Loss: 0.0341234132648
('Epoch: ', 72, '/', 500000)
-----------
Training Loss: 0.0202796645463
('Epoch: ', 73, '/', 500000)
-----------
Training Loss: 0.0133989294991
('Epoch: ', 74, '/', 500000)
-----------
Training Loss: 0.0186315272003
('Epoch: ', 75, '/', 500000)
-----------
Training Loss: 0.0194589998573
Model saved
('Epoch: ', 76, '/', 500000)
-----------
Training Loss: 0.0257925819606
('Epoch: ', 77, '/', 500000)
-----------
Training Loss: 0.0239839423448
('Epoch: ', 78, '/', 500000)
-----------
Training Loss: 0.0253231935203
('Epoch: ', 79, '/', 500000)
-----------
Training Loss: 0.0279013793916
('Epoch: ', 80, '/', 500000)
-----------
Training Loss: 0.0398402996361
Model saved
('Epoch: ', 81, '/', 500000)
-----------
Training Loss: 0.0412427783012
('Epoch: ', 82, '/', 500000)
-----------
Training Loss: 0.0410296469927
('Epoch: ', 83, '/', 500000)
-----------
Training Loss: 0.0227974969894
('Epoch: ', 84, '/', 500000)

Model saved
('Epoch: ', 141, '/', 500000)
-----------
Training Loss: 0.100275360048
('Epoch: ', 142, '/', 500000)
-----------
Training Loss: 0.130968853831
('Epoch: ', 143, '/', 500000)
-----------
Training Loss: 0.0382826812565
('Epoch: ', 144, '/', 500000)
-----------
Training Loss: 0.0242753438652
('Epoch: ', 145, '/', 500000)
-----------
Training Loss: 0.015976767987
Model saved
('Epoch: ', 146, '/', 500000)
-----------
Training Loss: 0.0521802082658
('Epoch: ', 147, '/', 500000)
-----------
Training Loss: 0.0824323967099
('Epoch: ', 148, '/', 500000)
-----------
Training Loss: 0.0337364673615
('Epoch: ', 149, '/', 500000)
-----------
Training Loss: 0.0589194335043
('Epoch: ', 150, '/', 500000)
-----------
Training Loss: 0.0683496743441
Model saved
('Epoch: ', 151, '/', 500000)
-----------
Training Loss: 0.0281406287104
('Epoch: ', 152, '/', 500000)
-----------
Training Loss: 0.048518396914
('Epoch: ', 153, '/', 500000)
-----------
Training Loss: 0.0254284869879
('Epoch: ', 154, '/

Training Loss: 0.0207612942904
Model saved
('Epoch: ', 211, '/', 500000)
-----------
Training Loss: 0.0164556633681
('Epoch: ', 212, '/', 500000)
-----------
Training Loss: 0.0136061357334
('Epoch: ', 213, '/', 500000)
-----------
Training Loss: 0.0188297796994
('Epoch: ', 214, '/', 500000)
-----------
Training Loss: 0.0177315026522
('Epoch: ', 215, '/', 500000)
-----------
Training Loss: 0.0128339603543
Model saved
('Epoch: ', 216, '/', 500000)
-----------
Training Loss: 0.0180342234671
('Epoch: ', 217, '/', 500000)
-----------
Training Loss: 0.0177926514298
('Epoch: ', 218, '/', 500000)
-----------
Training Loss: 0.0248466972262
('Epoch: ', 219, '/', 500000)
-----------
Training Loss: 0.0172574184835
('Epoch: ', 220, '/', 500000)
-----------
Training Loss: 0.01680983603
Model saved
('Epoch: ', 221, '/', 500000)
-----------
Training Loss: 0.0164918079972
('Epoch: ', 222, '/', 500000)
-----------
Training Loss: 0.0184681303799
('Epoch: ', 223, '/', 500000)
-----------
Training Loss: 0.

('Epoch: ', 280, '/', 500000)
-----------
Training Loss: 0.0157120637596
Model saved
('Epoch: ', 281, '/', 500000)
-----------
Training Loss: 0.0159491859376
('Epoch: ', 282, '/', 500000)
-----------
Training Loss: 0.0242701005191
('Epoch: ', 283, '/', 500000)
-----------
Training Loss: 0.0240984596312
('Epoch: ', 284, '/', 500000)
-----------
Training Loss: 0.0205650255084
('Epoch: ', 285, '/', 500000)
-----------
Training Loss: 0.0201418902725
Model saved
('Epoch: ', 286, '/', 500000)
-----------
Training Loss: 0.0236600004137
('Epoch: ', 287, '/', 500000)
-----------
Training Loss: 0.0167877562344
('Epoch: ', 288, '/', 500000)
-----------
Training Loss: 0.0150578776374
('Epoch: ', 289, '/', 500000)
-----------
Training Loss: 0.015222478658
('Epoch: ', 290, '/', 500000)
-----------
Training Loss: 0.019182998687
Model saved
('Epoch: ', 291, '/', 500000)
-----------
Training Loss: 0.0165339335799
('Epoch: ', 292, '/', 500000)
-----------
Training Loss: 0.0154531812295
('Epoch: ', 293, 

...

('Epoch: ', 16363, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16364, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16365, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16366, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16367, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16368, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16369, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16370, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16371, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16372, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16373, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16374, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16375, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16376, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16377, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16378, 

('Epoch: ', 16439, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16440, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16441, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16442, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16443, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16444, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16445, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16446, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16447, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16448, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16449, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16450, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16451, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16452, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16453, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16454, 

('Epoch: ', 16515, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16516, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16517, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16518, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16519, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16520, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16521, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16522, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16523, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16524, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16525, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16526, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16527, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16528, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16529, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16530, 

Model saved
('Epoch: ', 16591, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16592, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16593, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16594, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16595, '/', 500000)
-----------
Training Loss: 0.0
Model saved
('Epoch: ', 16596, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16597, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16598, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16599, '/', 500000)
-----------
Training Loss: 0.0
('Epoch: ', 16600, '/', 500000)
-----------
Training Loss: 0.0
Model saved


KeyboardInterrupt: 

## Step 8: Watch our Agent play 👀
Now that we trained our agent, we can test it

In [20]:
saver = tf.train.Saver()
num_of_episodes = 3

with tf.Session() as sess:
    game = create_environment(visible=True)
    
    saver.restore(sess, "./models/modelPGN.ckpt")
    
    for i in range(num_of_episodes):
        game.new_episode()

        state = game.get_state().screen_buffer
        state, stacked_frames = stack_frames(state, is_new_episode=True)

        while not game.is_episode_finished():
        
            # Run state though policy and calculate action
            action_probability_distribution = sess.run(PGNetwork.action_distribution, 
                                                       feed_dict={PGNetwork.inputs_: state.reshape(1, *STATE_SIZE)})

            # Sample action with respect to action probabilities
            action = np.random.choice(range(action_probability_distribution.shape[1]), 
                                      p=action_probability_distribution.ravel())
            action = POSSIBLE_ACTIONS[action]

            # Perform action
            reward = game.make_action(action)
            done = game.is_episode_finished()
            time.sleep(0.007)
            if done:
                break
            else:
                # If not done, the next state becomes the current state
                next_state = game.get_state().screen_buffer
                next_state, stacked_frames = stack_frames(next_state, is_new_episode=False, stacked_frames=stacked_frames)
                state = next_state
        time.sleep(1)
        

        print("Score for episode ", i, " :", game.get_total_reward())
    game.close()

ViZDoomErrorException: Could not initialize SDL video:
Failed to connect to the Mir Server


In [36]:
!apt-cache search libsdl

libsdl1.2-dev - Simple DirectMedia Layer development files
libsdl1.2debian - Simple DirectMedia Layer
erlang-esdl - Erlang bindings to the Simple Direct Media Library
libsdl-console - Console that can be added to any SDL application, libraries
libsdl-console-dev - Console that can be added to any SDL application, development files
libsdl-gfx1.2-5 - drawing and graphical effects extension for SDL
libsdl-gfx1.2-dev - development files for SDL_gfx
libsdl-gfx1.2-doc - documentation files for SDL_gfx
libsdl-gst - SDL bindings for GNU Smalltalk
libsdl-image-gst - SDL_image bindings for GNU Smalltalk
libsdl-image1.2 - Image loading library for Simple DirectMedia Layer 1.2, libraries
libsdl-image1.2-dbg - Image loading library for Simple DirectMedia Layer 1.2, debugging
libsdl-image1.2-dev - Image loading library for Simple DirectMedia Layer 1.2, development files
libsdl-mixer-gst - SDL_mixer bindings for GNU Smalltalk
libsdl-mixer1.2 - Mixer library for Simple DirectMedia Layer 1.2, libraries